In [1]:
import numpy as np
import pandas as pd
import matplotlib
from tqdm import tqdm 
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import math
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn import preprocessing

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost.sklearn import XGBClassifier

from config import base_algo_print_names, base_algo_symbols, ensemble_algo_print_names, cv_split_sets

# styling:
import seaborn as sns
plt.style.use(['ggplot'])
sns.set_palette("deep")

matplotlib.rcParams["figure.dpi"] = 300


In [2]:
# some global variables

min_max_scaler = preprocessing.MinMaxScaler()
algo_names = base_algo_print_names
algos = base_algo_symbols
# ensemble_algo_names = ["OutPredict", 'Inferelator', 'GRNBoost2', 'Genie3', 'PIDC', 'LEAP', 'SCODE', 'SCRIBE', 'SINCERITIES', 'PPCOR', 'Ensemble A', 'Ensemble B', 'Ensemble C', 'Ensemble D']
ensemble_algo_names = ensemble_algo_print_names

In [ ]:
# some filtering on the network, only validated tfs are kept
set_dir = 'tf_split_sets/42/'
train_df = pd.read_csv(set_dir+'train_by_tf_genes.csv', index_col=0)
test_df = pd.read_csv(set_dir+'test_by_tf_genes.csv', index_col=0)
# op_mat = pd.read_csv(set_dir+'Matrix_TF_gene_best_model.tsv', sep='\t', index_col=0)
# inf_mat = pd.read_csv(set_dir+'inf_matrix_tf_{}.tsv'.format(set_serial), sep='\t', index_col=0)

gs_tf_set = set()
for index, row in test_df.iterrows():
    edge_name = index
    regulator_name, target_name = edge_name.split('>')
    if (int(row['edge_exist']) == 1):
        gs_tf_set.add(regulator_name)
for index, row in train_df.iterrows():
    edge_name = index
    regulator_name, target_name = edge_name.split('>')
    if (int(row['edge_exist']) == 1):
        gs_tf_set.add(regulator_name)

In [4]:
# staking run on a particular split and a particular ensemble classifier
def evaluate_run(set_serial, clf):
    set_dir = 'tf_split_sets/{}/'.format(set_serial)
    train_df = pd.read_csv(set_dir+'train_by_tf_genes.csv', index_col=0)
    test_df = pd.read_csv(set_dir+'test_by_tf_genes.csv', index_col=0)
    
    to_drop_list = []
    for index, row in test_df.iterrows():
        edge_name = index
        regulator_name, target_name = edge_name.split('>')
        if (regulator_name not in gs_tf_set):
            to_drop_list.append(index)
    test_df = test_df.drop(to_drop_list)
    to_drop_list = []
    for index, row in train_df.iterrows():
        edge_name = index
        regulator_name, target_name = edge_name.split('>')
        if (regulator_name not in gs_tf_set):
            to_drop_list.append(index)
    train_df = train_df.drop(to_drop_list)

    min_max_scaler = preprocessing.MinMaxScaler()
    X_train = train_df[['op', 'inf', 'grnboost', 'genie', 'pidc', 'leap', 'scode', 'scribe', 'sincerities', 'ppcor']].values
    X_train = min_max_scaler.fit_transform(X_train)
    y_train = train_df['edge_exist']
    X_test = test_df[['op', 'inf', 'grnboost', 'genie', 'pidc', 'leap', 'scode', 'scribe', 'sincerities', 'ppcor']].values
    X_test = min_max_scaler.fit_transform(X_test)
    y_test = test_df['edge_exist']
    clf.fit(X_train, y_train)
    ensemble_pr = precision_recall_curve(y_test, clf.predict_proba(X_test)[:,1])

    X_train = train_df[['op', 'grnboost', 'genie', 'pidc', 'leap', 'sincerities', 'ppcor']].values
    X_train = min_max_scaler.fit_transform(X_train)
    y_train = train_df['edge_exist']
    X_test = test_df[['op', 'grnboost', 'genie', 'pidc', 'leap', 'sincerities', 'ppcor']].values
    X_test = min_max_scaler.fit_transform(X_test)
    y_test = test_df['edge_exist']
    clf.fit(X_train, y_train)
    ensemble_pr_less = precision_recall_curve(y_test, clf.predict_proba(X_test)[:,1])

    # X_train = train_df[['op', 'grnboost', 'genie', 'pidc', 'leap', 'sincerities', 'ppcor', 'lagged_cor']].values
    # X_train = min_max_scaler.fit_transform(X_train)
    # y_train = train_df['edge_exist']
    # X_test = test_df[['op', 'grnboost', 'genie', 'pidc', 'leap', 'sincerities', 'ppcor', 'lagged_cor']].values
    # X_test = min_max_scaler.fit_transform(X_test)
    # y_test = test_df['edge_exist']
    # clf.fit(X_train, y_train)
    # ensemble_pr_less_priors = precision_recall_curve(y_test, clf.predict_proba(X_test)[:,1])

    X_train = train_df[['op', 'inf', 'grnboost', 'genie', 'pidc', 'leap', 'scode', 'scribe', 'sincerities', 'ppcor', 'ss_cor', 'lagged_cor']].values
    X_train = min_max_scaler.fit_transform(X_train)
    y_train = train_df['edge_exist']
    X_test = test_df[['op', 'inf', 'grnboost', 'genie', 'pidc', 'leap', 'scode', 'scribe', 'sincerities', 'ppcor', 'ss_cor', 'lagged_cor']].values
    X_test = min_max_scaler.fit_transform(X_test)
    y_test = test_df['edge_exist']
    # clf.fit(X_train, y_train)
    # ensemble_pr_priors = precision_recall_curve(y_test, clf.predict_proba(X_test)[:,1])

    rand_aupr = y_test.sum()/len(y_test)
    algo_names = ["OutPredict", 'Inferelator', 'GRNBoost2', 'Genie3', 'PIDC', 'LEAP', 'SCODE', 'SCRIBE', 'SINCERITIES', 'PPCOR']
    pr_list = []
    pr_scores = []
    pr_ratios = []
    for i in range(10):
        pr = precision_recall_curve(y_test, X_test[:,i])
        pr_list.append(pr)
        pr_scores.append(auc(pr[1], pr[0]))
        pr_ratios.append(auc(pr[1], pr[0])/rand_aupr)
    rankings = np.argsort(np.array(pr_scores))[::-1]

    pr_ratios.append(auc(ensemble_pr[1], ensemble_pr[0])/rand_aupr)
    pr_ratios.append(auc(ensemble_pr_less[1], ensemble_pr_less[0])/rand_aupr)
    # pr_ratios.append(auc(ensemble_pr_less_priors[1], ensemble_pr_less_priors[0])/rand_aupr)
    # pr_ratios.append(auc(ensemble_pr_priors[1], ensemble_pr_priors[0])/rand_aupr)
    aupr_df.loc[len(aupr_df.index)] = pr_ratios

In [5]:
# stacking with Naive Bayes with Gaussian kernel
clf = GaussianNB(var_smoothing=2.848035868435805e-09)
aupr_df = pd.DataFrame(columns=ensemble_algo_names)
for i in tqdm(cv_split_sets):
    evaluate_run(i, clf)
aupr_df.to_csv('arabidopsis_auprc_nb.csv', index=False)

100%|██████████| 20/20 [02:49<00:00,  8.46s/it]


In [6]:
# stacking with random forest
clf = RandomForestClassifier(bootstrap=False, max_depth=8, max_features='sqrt', n_estimators=500, n_jobs=-1, random_state=42)
aupr_df = pd.DataFrame(columns=ensemble_algo_names)
for i in tqdm(cv_split_sets):
    evaluate_run(i, clf)
aupr_df.to_csv('arabidopsis_auprc_rf.csv', index=False)

100%|██████████| 20/20 [06:01<00:00, 18.10s/it]


In [5]:
# stacking with logistic regression
clf = LogisticRegression(random_state=42, n_jobs=-1, C=100, penalty='l2',solver='newton-cg')
aupr_df = pd.DataFrame(columns=ensemble_algo_names)
for i in tqdm(cv_split_sets):
    evaluate_run(i, clf)
aupr_df.to_csv('arabidopsis_auprc_lr.csv', index=False)

100%|██████████| 20/20 [03:01<00:00,  9.09s/it]


In [ ]:
# stacking with AB boost
clf = AdaBoostClassifier(random_state=42, learning_rate=0.1, n_estimators=100)
aupr_df = pd.DataFrame(columns=ensemble_algo_names)
for i in tqdm(cv_split_sets):
    evaluate_run(i, clf)
aupr_df.to_csv('arabidopsis_auprc_ab.csv', index=False)